In [ ]:
import pandas as pd
import parsers
import plans
from matplotlib import pyplot as plt

In [ ]:
df = parsers.read_pazgaz_report('דוח צריכה לירון אילוז 345374663 אפריל-יוני.xlsx')
df['day_name'] = df.index.day_name()
df.consumption.plot()

In [ ]:
default_rate = 0.6

In [ ]:
total_consumption = df.consumption.sum()
print("Total consumption: %d KW"%(total_consumption))
print('Cost: %.2f NIS' %(total_consumption * default_rate))

In [ ]:
def apply_plan_discount(df, plan):
    total_consumption = df.consumption.sum()
    discunted_consumption = 0
    for period in plan['periods']:
        discunted_consumption += df[df.day_name == period['day']].between_time(period['start'].time(), period['end'].time()).consumption.sum()
    
    discount = discunted_consumption * plan['discount'] * default_rate
    print(f"Plan: {plan['name']}")
    # print("Discunted consumption: %d KW" %(discunted_consumption))
    print("Saving: %.2f NIS" % (discount))
    print("")
    return discount

In [ ]:
discounts = []
for provider in plans.provider_plans:
    print(provider['provider'])
    for plan in provider['plans']:
        d = apply_plan_discount(df, plan)
        discounts.append({"provider": provider['provider'], "plan": plan['name'], "discount": d})

In [ ]:
pd.DataFrame(discounts).sort_values(by='discount', ascending=False).reset_index(drop=True)